# Colab Setup

In [0]:
!nvidia-smi

Sat Sep 28 11:00:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install gdown --quiet

# Imports and Data Setup

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import numpy as np  
import pandas as pd
from pathlib import Path
import scipy.io
import h5py

import sklearn.metrics as metrics
import matplotlib.pyplot as plt

from scipy.signal import resample

from scipy.signal import find_peaks as findPeaks
from scipy import sparse
from scipy.sparse.linalg import spsolve

from pathlib import Path
import gdown

In [0]:
datapath = Path('./data')

In [0]:
# Download Raw Data
inertial_url = 'https://drive.google.com/uc?export=download&id=1gGd3nq889m9iU8gkyqk1YHLOqmR31VWb'
inertial_file = 'Inertial.zip'

skeleton_url = 'https://drive.google.com/uc?export=download&id=1rvnhTitWIWF-9biYyzzGXXcaY5BXecV1'
skeleton_file = 'Skeleton.zip'

gdown.download(inertial_url, inertial_file, quiet=True)
gdown.download(skeleton_url, skeleton_file, quiet=True)

'Skeleton.zip'

In [0]:
!unzip -qq ./Inertial.zip -d ./data/

In [0]:
!unzip -qq ./Skeleton.zip -d ./data/

# Process Dataset

## Process Skeleton Data

In [0]:
def import_data(file_type, action, subject, trial):
    filename = datapath/f'{file_type}/a{action}_s{subject}_t{trial}_{file_type.lower()}.mat'
    if Path(filename).is_file():
        mat = scipy.io.loadmat(filename)
        if file_type.lower() == 'inertial':
            return mat['d_iner']
        else:
            return mat['d_skel']
    else:
        return None

In [0]:
def calculate_angle(point1,point2,point3):


    point2_1 = point1 - point2
    point2_3 = point3 - point2

    cosine_angle = np.dot(point2_1, point2_3) / (np.linalg.norm(point2_1) * np.linalg.norm(point2_3))
    angle = np.arccos(cosine_angle)
    angle = np.degrees(angle)

    return angle

In [0]:
# Importing Skeleton Data Variance
def process_skeleton_angles():

    reshaped_data = np.zeros([861,116,60])
    trDat = []
    trLbl = []
    tsDat = []
    tsLbl = []
    
    counter = 0
    subject = 0
    angle = 30
    duration = 1.5

    for i in range (864):
        
        if counter == 32:
            counter = 0
        subject = int((counter-counter%4)/4+1)
        counter = counter + 1
        action = int((i-i%32)/32+1)
        take = i%4+1
        
        
        data = import_data('Skeleton',action,subject,take)
        
        if data is not None and action in[1,7,21,22,27]:
                        
            temp_array = np.zeros([data.shape[2],60])
            temp_array_angle = np.zeros([16,data.shape[2]])
            
            
            for j in range (data.shape[2]):
                temp_array[j,:] = data[:,:,j].flatten()
                
                temp_array_angle[0,j]= 180 - calculate_angle(temp_array[j,0:2],temp_array[j,3:5],temp_array[j,6:8]) #head_spine_angle
                temp_array_angle[1,j]= 180 - calculate_angle(temp_array[j,3:5],temp_array[j,6:8],temp_array[j,9:11]) #spine_angle
                
                temp_array_angle[2,j]= 180 - calculate_angle(temp_array[j,15:17],temp_array[j,18:20],temp_array[j,21:23]) #left_wrist_angle
                temp_array_angle[3,j]= 180 - calculate_angle(temp_array[j,13:14],temp_array[j,15:17],temp_array[j,18:20]) #left_elbow_angle
                temp_array_angle[4,j]= 180 - calculate_angle(temp_array[j,3:5],temp_array[j,12:14],temp_array[j,15:17]) #left_shoulder_angle
                temp_array_angle[5,j]= 180 - calculate_angle(temp_array[j,39:41],temp_array[j,42:44],temp_array[j,45:47]) #left_ankle_angle
                temp_array_angle[6,j]= 180 - calculate_angle(temp_array[j,36:38],temp_array[j,39:41],temp_array[j,42:44]) #left_knee_angle
                temp_array_angle[7,j]= 180 - calculate_angle(temp_array[j,9:11],temp_array[j,36:38],temp_array[j,39:41]) #left_hip_angle
                temp_array_angle[8,j]= 180 - calculate_angle(temp_array[j,0:2],temp_array[j,3:5],temp_array[j,12:14]) #left_shoulder_head_angle
                temp_array_angle[9,j]= 180 - calculate_angle(temp_array[j,27:29],temp_array[j,30:32],temp_array[j,33:35]) #right_wrist_angle
                temp_array_angle[10,j]= 180 - calculate_angle(temp_array[j,24:26],temp_array[j,27:29],temp_array[j,30:32]) #right_elbow_angle
                temp_array_angle[11,j]= 180 - calculate_angle(temp_array[j,3:5],temp_array[j,24:26],temp_array[j,27:29]) #right_shoulder_angle
                temp_array_angle[12,j]= 180 - calculate_angle(temp_array[j,51:53],temp_array[j,54:56],temp_array[j,57:59]) #right_ankle_angle
                temp_array_angle[13,j]= 180 - calculate_angle(temp_array[j,48:50],temp_array[j,51:53],temp_array[j,54:56]) #right_knee_angle
                temp_array_angle[14,j]= 180 - calculate_angle(temp_array[j,9:11],temp_array[j,48:50],temp_array[j,51:53]) #right_hip_angle
                temp_array_angle[15,j]= 180 - calculate_angle(temp_array[j,0:2],temp_array[j,3:5],temp_array[j,24:26]) #right_shoulder_head_angle

      
            max_length = 0
            number_of_peaks = 0
            list_of_peaks = []
            peak_sequence = []
            
            #Make a list of peaks
            for k in range (16):

                J1= temp_array_angle[k]
                (allPks,_) = findPeaks(J1,prominence=angle,width=duration)
                number_of_peaks = number_of_peaks + len(allPks)
                if len(allPks) > max_length:
                    max_length = len(allPks)
                    
                for l in range (len(allPks)):
                    list_of_peaks.append(allPks[l])
                
            list_of_peaks.sort()
            peaks = np.zeros([16,max_length])
            
            peak_sequence = list_of_peaks.copy()
            
            for k in range (len(peak_sequence)):
                peak_sequence[k] = 17
            
            
            #make peak sequence
            for k in range (16):
                J1= temp_array_angle[k]
                (allPks,_) = findPeaks(J1,prominence=angle,width=duration)
                if len(allPks) >= 1:
                    for l in range (len(allPks)):
                        for m in range (len(list_of_peaks)):
                            if allPks[l]==list_of_peaks[m] and peak_sequence[m] == 17:
                                peak_sequence[m] = k
                                break
       
            #Tranform peak sequence to alphabet
            for k in range (len(peak_sequence)):
                if peak_sequence[k] == 0:
                    peak_sequence[k] = 'A'
                elif peak_sequence[k] == 1:
                    peak_sequence[k] = 'B'
                elif peak_sequence[k] == 2:
                    peak_sequence[k] = 'C'
                elif peak_sequence[k] == 3:
                    peak_sequence[k] = 'D'
                elif peak_sequence[k] == 4:
                    peak_sequence[k] = 'E'
                elif peak_sequence[k] == 5:
                    peak_sequence[k] = 'F'
                elif peak_sequence[k] == 6:
                    peak_sequence[k] = 'G'
                elif peak_sequence[k] == 7:
                    peak_sequence[k] = 'H'
                elif peak_sequence[k] == 8:
                    peak_sequence[k] = 'I'
                elif peak_sequence[k] == 9:
                    peak_sequence[k] = 'J'
                elif peak_sequence[k] == 10:
                    peak_sequence[k] = 'K'
                elif peak_sequence[k] == 11:
                    peak_sequence[k] = 'L'
                elif peak_sequence[k] == 12:
                    peak_sequence[k] = 'M'
                elif peak_sequence[k] == 13:
                    peak_sequence[k] = 'N'
                elif peak_sequence[k] == 14:
                    peak_sequence[k] = 'O'
                elif peak_sequence[k] == 15:
                    peak_sequence[k] = 'P'
                else:
                    peak_sequence[k] = 'X'
                                                             
            
            #make peak array
            for k in range (16):    
                J1= temp_array_angle[k]
                (allPks,_) = findPeaks(J1,prominence=angle,width=duration)
                if len(allPks) >=1:
                    for m in range(len(allPks)):
                        peaks[k][m]=allPks[m]


                
            if subject in [2, 4, 6, 8]:
                tsDat.append(peak_sequence.copy())
              
                tsLbl.append(action-1)
            elif subject in [1, 3, 5, 7]:
                trDat.append(peak_sequence.copy())
                trLbl.append(action-1)
 
   
    trDat = np.array(trDat)
    trLbl = np.array(trLbl)
    tsDat = np.array(tsDat)
    tsLbl = np.array(tsLbl)
    
    return trDat, trLbl, tsDat, tsLbl

trDat, trLbl, tsDat, tsLbl = process_skeleton_angles()

print(trDat.shape)
print(trLbl.shape)
print(tsDat.shape)
print(tsLbl.shape)

(80,)
(80,)
(79,)
(79,)


In [0]:
Graph1 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}
          


Graph2 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph3 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph4 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph5 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph6 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph7 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph8 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph9 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph10 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph11 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph12 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph13 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph14 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph15 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph16 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph17 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph18 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph19 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph20 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph21 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph22 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph23 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph24 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph25 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph26 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

Graph27 = {'A':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'B':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'C':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'D':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'E':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'F':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'G':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'H':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'I':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'J':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'K':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'L':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'M':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'N':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'O':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0},
          'P':{'A0':0,'B0':0,'C0':0,'D0':0,'E0':0,'F0':0,'G0':0,'H0':0,'I0':0,'J0':0,'K0':0,'L0':0,'M0':0,'N0':0,'O0':0,'P0':0,
          'A1':0,'B1':0,'C1':0,'D1':0,'E1':0,'F1':0,'G1':0,'H1':0,'I1':0,'J1':0,'K1':0,'L1':0,'M1':0,'N1':0,'O1':0,'P1':0,
          'A2':0,'B2':0,'C2':0,'D2':0,'E2':0,'F2':0,'G2':0,'H2':0,'I2':0,'J2':0,'K2':0,'L2':0,'M2':0,'N2':0,'O2':0,'P2':0}}

GraphModel = [Graph1,Graph2,Graph3,Graph4,Graph5,Graph6,Graph7,Graph8,Graph9,Graph10,Graph11,Graph12,Graph13,Graph14,Graph15,
             Graph16,Graph17,Graph18,Graph19,Graph20,Graph21,Graph22,Graph23,Graph24,Graph25,Graph26,Graph27]
GraphScore = np.zeros([27])


       
    


In [0]:
for i in range (len(trDat)):
    sequence = trDat[i]
    GraphScore[trLbl[i]] += len(sequence)
    visit_record = {'A':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'B':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'C':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'D':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'E':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'F':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'G':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'H':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'I':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'J':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'K':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'L':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'M':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'N':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'O':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'P':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0}}
    
    for j in range (len(sequence)-1):
        next_node = f'{sequence[j+1]}{visit_record[sequence[j]][sequence[j+1]]}'
        visit_record[sequence[j]][sequence[j+1]] += 1
        if  visit_record[sequence[j]][sequence[j+1]] < 4:
            GraphModel[trLbl[i]][sequence[j]][next_node] += 1

In [0]:
class_score = np.zeros([len(tsDat),27])

for i in range (len(tsDat)):
    sequence = tsDat[i]
    
    for k in range (27):
        visit_record = {'A':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'B':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'C':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'D':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'E':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'F':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'G':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'H':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'I':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'J':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'K':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'L':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'M':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'N':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'O':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0},
        'P':{'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0}}
        
        combo_count = 0
        for j in range (len(sequence)-1):
            next_node = f'{sequence[j+1]}{visit_record[sequence[j]][sequence[j+1]]}'
            visit_record[sequence[j]][sequence[j+1]] += 1
            if  visit_record[sequence[j]][sequence[j+1]] < 4:
                class_score[i][k] += GraphModel[k][sequence[j]][next_node]
            else:
                class_score[i][k] += 1 #plus 1 for cases that goes through the same node for more than 3 times
            
            if  visit_record[sequence[j]][sequence[j+1]] < 4:
                if GraphModel[k][sequence[j]][next_node] != 0:
                    combo_count += 1
                else:
                    combo_count = 0
                if combo_count > 1:
                    class_score[i][k] += combo_count*2 #Hyper parameter
        
        #Hyper parameter - apply scaling to score for similarity/difference in sequence length
        average_length = GraphScore[k]/16
        if len(sequence) > average_length:
            class_score[i][k] = class_score[i][k]*average_length/len(sequence)
        else:
            class_score[i][k] = class_score[i][k]*len(sequence)/average_length


In [0]:
y_pred = []

for i in range (len(tsDat)):
    largest_number = 0
    class_no = 0
    for j in range (27):
        if class_score[i][j] > largest_number:
            largest_number = class_score[i][j]
            class_no = j
    y_pred.append(class_no)

y_pred = np.array(y_pred)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(tsLbl,y_pred))  
print(" ")
print(classification_report(tsLbl,y_pred))  
print('Accuracy: %.2f%%'%(accuracy_score(tsLbl, y_pred)*100)) 

[[16  0  0  0  0]
 [ 0 14  0  0  2]
 [ 0  0 13  1  2]
 [ 0  0  0 16  0]
 [ 0  0  0  1 14]]
 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           6       1.00      0.88      0.93        16
          20       1.00      0.81      0.90        16
          21       0.89      1.00      0.94        16
          26       0.78      0.93      0.85        15

    accuracy                           0.92        79
   macro avg       0.93      0.92      0.92        79
weighted avg       0.94      0.92      0.92        79

Accuracy: 92.41%
